<a href="https://colab.research.google.com/github/kpmadabhu/EEE598_proj_tuh_eeg/blob/main/EEE598_Proj_TUH_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TUSZ Preprocessing Pipeline – Smart Canonical Montage (Reference Alignment)

## High-level
This notebook builds a seizure vs background window dataset from TUH Seizure Corpus (TUSZ) tcp-bipolar EEG.

It does:
- Recursively walks `train/`, `dev/`, or `eval` under `ROOT_DIR`.
- For each EDF:
  - Load EEG + its seizure annotations (.csv).
  - Bandpass 0.5–40 Hz using a Kaiser FIR, zero-phase (`filtfilt`).
  - Downsample to 250 Hz *if* original fs is higher; never upsample.
  - Robust z-score per channel (median/IQR).
  - Slice into 10 s windows with 5 s hop.
  - Label each window seizure(1) or background(0), pooling all seizure subtypes.
- Skips recordings that are too short for `filtfilt` padding.
- Saves:
  - `split_windows.npz` with windows, labels, and metadata.
  - `qc_window.png` plotting an informative example window.
  - `class_balance.png` showing seizure/background counts.

## Smart Canonical Montage
We do **not** force a giant hard-coded canonical bipolar channel list upfront.
Instead we learn a canonical layout from *inside the split*:

1. The **first usable recording** becomes our **reference layout**.
   - We store its channel names/ordering. Call this `ref_ch_names`.
2. Every later recording is aligned to that same reference layout:
   - For each reference channel name (like `FP1-F7`), we try to find the best-matching channel name in the new recording using a robust normalizer.
   - If found, we copy that channel.
   - If missing, we fill zeros in that row.
3. We compute a **coverage ratio** = fraction of reference channels we could actually fill with real data from this recording.
   - If coverage is too low (default <30%), we SKIP that recording to avoid mostly-zero data.

So we get:
- Consistent `[C, T]` across all recordings (good for training).
- Mostly real EEG, not all zeros.
- QC plot that reflects the aligned tensors you’ll actually train on.

## Prediction Labeling

- **Positive (y = 1)**: **pre-ictal** windows only — the interval *before onset* where a predictor should alarm.  
  Defined per seizure as **[onset − SOP_MIN, onset − SPH_MIN]**.
- **Negative (y = 0)**: **interictal** background **plus** a short **early-ictal** portion immediately after onset:  
  **[onset, min(onset + EARLY_ICTAL_KEPT_S, offset)]**.
- **Ignored** (not used for training): the **prediction horizon** just before onset **(onset − SPH_MIN, onset)**, the **remainder of ictal** after the early-ictal portion, and an optional **post-ictal** buffer **(offset, offset + POSTICTAL_BUFFER_S]** when enabled.
- **Artifacts**: windows overlapping artifact intervals are **dropped**; if `KEEP_PREICTAL_EVEN_IF_ARTIFACT = True`, pre-ictal windows are **kept** even when overlapping artifacts (positives are scarce).

### Timing variables (all in seconds)
- `SOP_MIN` — Seizure Occurrence Period look-back (e.g., 30*60).  
- `SPH_MIN` — Seizure Prediction Horizon “no-alarm” band (e.g., 5*60).  
- `EARLY_ICTAL_KEPT_S` — early-ictal kept as **negative** (e.g., 10).  
- `POSTICTAL_BUFFER_S` — optional ignore after offset (0 disables).  
- `KEEP_PREICTAL_EVEN_IF_ARTIFACT` — keep/don’t keep pre-ictal windows that overlap artifacts.

### Note on normalization (robust z)
Pre-ictal windows can be very quiet (IQR≈0). Robust z-score uses a **safe denominator**  
`denom = max(IQR, 0.1*STD, 1e-6)` to prevent extreme amplitudes while preserving behavior elsewhere.

## Runtime knobs
- `MAX_SUBJECTS`: cap how many subjects to include (fast debug).
- `MAX_RECORDINGS`: cap recordings per subject (or overall).
- `NUM_TAPS`: FIR length. Shorter => fewer clips skipped for being "too short".
- `GROUP_MODE`: iterate by `"subject"` (recommended) or just all EDFs (`"recording"`).
- `min_coverage`: skip a recording if < this fraction of reference channels could be mapped.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import mne
from scipy.signal import firwin, filtfilt, resample_poly
from math import gcd
import io, re

# ------------------ USER CONFIG ------------------
# Point this at the root of the TUSZ split folders that contain train/dev/eval
ROOT_DIR = Path("../../../../../../../../../../../../TUH-Seizure-Corpus/edf").expanduser().resolve()

# Output directory for NPZ + plots
OUT_DIR  = Path("./tusz_windows").expanduser().resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Windowing params
WIN_LEN_S = 10.0   # seconds per window
HOP_S     = 5.0    # seconds hop (50% overlap)

# --- Prediction labeling policy (all times in SECONDS) ---
SOP_MIN = 30 * 60          # 30 minutes before seizure onset (pre-ictal start)
SPH_MIN = 5  * 60          # 5 minutes before onset (prediction horizon, ignored)
EARLY_ICTAL_KEPT_S = 10    # first 10 s of ictal treated as NEGATIVE (y=0)
POSTICTAL_BUFFER_S = 0     # ignore after offset (0 disables)
KEEP_PREICTAL_EVEN_IF_ARTIFACT = True  # keep pre-ictal even if overlapping artifacts

# DSP params
NUM_TAPS  = 1001    # Kaiser FIR taps for bandpass 0.5-40Hz (shorter -> fewer skips)
TARGET_FS = 250.0  # We'll downsample to 250 Hz if fs_orig > 250 Hz; never upsample

# Runtime limiting params
MAX_SUBJECTS   = 10        # limit number of subjects used (debug speed). None = no explicit cap
MAX_RECORDINGS = 10      # max recordings per subject / overall. None = no explicit cap
GROUP_MODE     = "subject" # "subject" or "recording"

# Seizure subtype labels we consider "seizure" (label=1)
# Everything else becomes background (label=0)
SEIZURE_LABELS = {
    'seiz','fnsz','gnsz','spsz','cpsz','absz',
    'tnsz','cnsz','tcsz','atsz','mysz'
}

# Artifact / noise labels in TUSZ annotations.
# Based on TUSZ event map, includes muscle, eye movement, chewing, shiver,
# electrode pops, and combos. :contentReference[oaicite:0]{index=0}
ARTIFACT_LABELS = {
    'artf',        # generic artifact
    'eyem',        # eye movement / blink
    'chew',        # chewing
    'shiv',        # shiver / tremor
    'musc',        # muscle artifact
    'elec',        # electrode pop / line noise
    # common multi-label combos found in TUSZ maps:
    'eyem_chew','eyem_shiv','eyem_musc','eyem_elec',
    'chew_shiv','chew_musc','chew_elec',
    'shiv_musc','shiv_elec',
    'musc_elec'
}

print("ROOT_DIR:", ROOT_DIR)
print("OUT_DIR:", OUT_DIR)
